### Installation

Run below cell to install the module:

In [2]:
!pip install -q git+https://github.com/sebastian-sz/efficientnet-lite-keras@main

In [3]:
import os

import tensorflow as tf
from efficientnet_lite import EfficientNetLiteB0


print(tf.__version__)

2.5.0


### Download example dataset

In this section we are going to download example dataset.

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz | tar xz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  218M  100  218M    0     0  69.6M      0  0:00:03  0:00:03 --:--:-- 69.6M


Remove the License file so it doesn't mess up directory structure:

In [5]:
!rm flower_photos/LICENSE.txt

Preview Class names:

In [6]:
!ls flower_photos

daisy  dandelion  roses  sunflowers  tulips


### Load the data:

In [7]:
DATA_PATH = "./flower_photos"
BATCH_SIZE = 32
TARGET_SIZE = (224, 224)


def preprocess_data(images, labels):
    images = (images - 127.00) / 128.00
    return images, labels


def augment_data(images, labels):
    return tf.image.random_flip_left_right(image), label

In [11]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory=DATA_PATH,
    batch_size=BATCH_SIZE,
    image_size=TARGET_SIZE,
    label_mode="categorical"
)


Found 3670 files belonging to 5 classes.


In [12]:
for image_batch, label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.shape)

(32, 224, 224, 3)
(32, 5)


### Train 

Let us fine tune Efficient Net Lite.

In [13]:
def build_model(num_classes=5):
    base_model = EfficientNetLiteB0(
        input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], 3),
        include_top=False,
        pooling="avg",
        weights="imagenet"
    )

    base_model.trainable=False

    return tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(num_classes, activation="softmax")
    ])

In [14]:
model = build_model()
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

model.summary()

14024704/14023984 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetlite0 (Functiona (None, 1280)              3413024   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 3,419,429
Trainable params: 6,405
Non-trainable params: 3,413,024
_________________________________________________________________


In [15]:
model.fit(
    dataset,
    epochs=1,
)

115/115 [==============================] - 151s 1s/step - loss: 1.1772 - accuracy: 0.5390
